# Задача за класификация на текст

В този модул ще започнем с проста задача за класификация на текст, базирана на **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** набора от данни: ще класифицираме заглавия на новини в една от 4 категории: Свят, Спорт, Бизнес и Наука/Технологии.

## Наборът от данни

За да заредим набора от данни, ще използваме **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Сега можем да достъпим тренировъчната и тестовата част на набора от данни, като използваме `dataset['train']` и `dataset['test']` съответно:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Нека отпечатаме първите 10 нови заглавия от нашия набор от данни:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Векторизация на текст

Сега трябва да преобразуваме текста в **числа**, които могат да бъдат представени като тензори. Ако искаме представяне на ниво думи, трябва да направим две неща:

* Използване на **токенизатор**, за да разделим текста на **токени**.
* Създаване на **речник** от тези токени.

### Ограничаване на размера на речника

В примера с набора от данни AG News, размерът на речника е доста голям — повече от 100 хиляди думи. По принцип, не ни трябват думи, които рядко се срещат в текста — само няколко изречения ще ги съдържат, и моделът няма да се научи от тях. Затова има логика да ограничим размера на речника до по-малко число, като подадем аргумент на конструктора на векторизатора:

И двата тези етапа могат да бъдат обработени с помощта на слоя **TextVectorization**. Нека създадем обекта на векторизатора и след това извикаме метода `adapt`, за да преминем през целия текст и да изградим речник:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Забележка**: Използваме само подмножество от целия набор от данни, за да изградим речник. Правим това, за да ускорим времето за изпълнение и да не ви караме да чакате. Въпреки това, поемаме риска, че някои от думите от целия набор от данни няма да бъдат включени в речника и ще бъдат игнорирани по време на обучението. Следователно, използването на целия размер на речника и преминаването през целия набор от данни по време на `adapt` би трябвало да увеличи крайната точност, но не значително.

Сега можем да достъпим действителния речник:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Използвайки векторизатора, можем лесно да кодираме всеки текст в набор от числа:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Представяне на текст чрез "чанта с думи"

Тъй като думите носят значение, понякога можем да разберем смисъла на даден текст, като просто разгледаме отделните думи, без да обръщаме внимание на техния ред в изречението. Например, при класифициране на новини, думи като *време* и *сняг* вероятно ще насочат към *прогноза за времето*, докато думи като *акции* и *долар* биха се отнасяли към *финансови новини*.

**Чанта с думи** (BoW) е най-простото за разбиране традиционно представяне на вектор. Всяка дума е свързана с индекс във вектора, а елементът на вектора съдържа броя на срещанията на всяка дума в даден документ.

![Изображение, показващо как представянето чрез чанта с думи се съхранява в паметта.](../../../../../translated_images/bg/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Можете също да мислите за BoW като сума от всички едно-горещо-кодирани вектори за отделните думи в текста.

По-долу е даден пример за това как да генерирате представяне чрез чанта с думи, използвайки библиотеката Scikit Learn на Python:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Можем също да използваме Keras векторизатора, който дефинирахме по-горе, като преобразуваме всяко число на дума в one-hot кодиране и събираме всички тези вектори.


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Забележка**: Може да се изненадате, че резултатът се различава от предишния пример. Причината е, че в примера с Keras дължината на вектора съответства на размера на речника, който е създаден от целия набор данни AG News, докато в примера със Scikit Learn създадохме речника от примерния текст в движение.


## Обучение на класификатора BoW

Сега, след като научихме как да създадем представянето на текста като "чанта с думи", нека обучим класификатор, който го използва. Първо, трябва да преобразуваме нашия набор от данни в представяне като "чанта с думи". Това може да се постигне чрез използване на функцията `map` по следния начин:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Сега нека дефинираме прост класификаторен невронен модел, който съдържа един линеен слой. Размерът на входа е `vocab_size`, а размерът на изхода съответства на броя на класовете (4). Тъй като решаваме задача за класификация, крайната активационна функция е **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Тъй като имаме 4 класа, точност над 80% е добър резултат.

## Обучение на класификатор като една мрежа

Тъй като векторизаторът също е слой на Keras, можем да дефинираме мрежа, която го включва, и да я обучим от край до край. По този начин не е необходимо да векторизираме набора от данни с помощта на `map`, а можем просто да подадем оригиналния набор от данни към входа на мрежата.

> **Note**: Все пак ще трябва да приложим `map` към нашия набор от данни, за да преобразуваме полетата от речници (като `title`, `description` и `label`) в кортежи. Въпреки това, когато зареждаме данни от диска, можем да изградим набор от данни с необходимата структура още от самото начало.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Биграми, триграми и n-грамове

Едно от ограниченията на подхода "чанта с думи" е, че някои думи са част от изрази с няколко думи. Например, думата „hot dog“ има напълно различно значение от думите „hot“ и „dog“ в други контексти. Ако винаги представяме думите „hot“ и „dog“ с едни и същи вектори, това може да обърка нашия модел.

За да се справим с това, често се използват **n-грамови представяния** в методите за класификация на документи, където честотата на всяка дума, дву-дума или три-дума е полезна характеристика за обучение на класификатори. При биграмовите представяния, например, ще добавим всички двойки думи към речника, в допълнение към оригиналните думи.

По-долу е даден пример как да се генерира биграмово представяне на чанта с думи, използвайки Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Основният недостатък на подхода с n-грам е, че размерът на речника започва да нараства изключително бързо. На практика е необходимо да комбинираме представянето с n-грам с техника за намаляване на размерността, като например *вграждания* (embeddings), които ще обсъдим в следващия модул.

За да използваме представяне с n-грам в нашия **AG News** набор от данни, трябва да подадем параметъра `ngrams` на конструктора на `TextVectorization`. Дължината на речника за биграми е **значително по-голяма** – в нашия случай тя е над 1.3 милиона токена! Затова има смисъл да ограничим и токените за биграми до някакъв разумен брой.

Можем да използваме същия код като по-горе, за да обучим класификатора, но това би било много неефективно по отношение на паметта. В следващия модул ще обучим класификатор за биграми, използвайки вграждания. Междувременно можете да експериментирате с обучението на класификатор за биграми в този ноутбук и да видите дали можете да постигнете по-висока точност.


## Автоматично изчисляване на BoW вектори

В примера по-горе изчислихме BoW вектори ръчно, като събрахме едноразрядните кодировки на отделните думи. Въпреки това, последната версия на TensorFlow ни позволява автоматично да изчисляваме BoW вектори, като предадем параметъра `output_mode='count` на конструктора на векторизатора. Това значително улеснява дефинирането и обучението на нашия модел:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Честота на термина - обратна честота на документа (TF-IDF)

В представянето на BoW, появите на думите се претеглят с една и съща техника, независимо от самата дума. Въпреки това е ясно, че често срещани думи като *a* и *in* са много по-малко важни за класификацията в сравнение със специализирани термини. В повечето задачи в обработката на естествен език някои думи са по-релевантни от други.

**TF-IDF** означава **честота на термина - обратна честота на документа**. Това е вариация на bag-of-words, при която вместо бинарна стойност 0/1, указваща появата на дума в документ, се използва стойност с плаваща запетая, която е свързана с честотата на появата на думата в корпуса.

По-формално, теглото $w_{ij}$ на дума $i$ в документ $j$ се дефинира като:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
където
* $tf_{ij}$ е броят на появите на $i$ в $j$, т.е. стойността на BoW, която видяхме по-рано
* $N$ е броят на документите в колекцията
* $df_i$ е броят на документите, съдържащи думата $i$ в цялата колекция

Стойността на TF-IDF $w_{ij}$ нараства пропорционално на броя пъти, в които една дума се появява в документ, и се компенсира от броя на документите в корпуса, които съдържат думата. Това помага да се коригира фактът, че някои думи се появяват по-често от други. Например, ако думата се появява във *всеки* документ в колекцията, $df_i=N$, и $w_{ij}=0$, и тези термини ще бъдат напълно игнорирани.

Можете лесно да създадете векторизация на текст с TF-IDF, използвайки Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

В Keras слоят `TextVectorization` може автоматично да изчислява TF-IDF честоти, като се зададе параметърът `output_mode='tf-idf'`. Нека повторим кода, който използвахме по-горе, за да видим дали използването на TF-IDF увеличава точността:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Заключение

Въпреки че представянията с TF-IDF предоставят тегла на честотата за различни думи, те не могат да представят значението или реда. Както известният лингвист Дж. Р. Фърт каза през 1935 г., "Пълното значение на една дума винаги е контекстуално, и никакво изследване на значението извън контекста не може да бъде взето на сериозно." По-късно в курса ще научим как да улавяме контекстуална информация от текст чрез моделиране на езика.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
